In [3]:
import pandas as pd
import funzioni as fx
lista_ticker = pd.read_parquet("lista_ticker.parquet")
lista_ticker.sample(frac=1)
tot_ticker = 1000
bilanciamento = 0.25
soglia_memoria = 14.5 #GB
X, Y = fx.set_di_tuning_pool(lista_ticker, tot_ticker, 'dati', f'_{tot_ticker}_{bilanciamento}', bilanciamento=bilanciamento)
print(X.shape)
print(Y.shape)

inizio=1000, n_simboli_addestramento=1000


Caricamento X e Y


KeyboardInterrupt: 